In [10]:
def g(df):
    for col in df.columns:
        if any(t in col for t in ["room_sp", "slab_sp", "thermal_comfort_data", "issues_count", "potential_savings"]):
            df.drop(col, axis=1, inplace=True)
    return df

In [11]:
import os
import json
import pandas as pd

year_folder = '../output/108/2023'
output_year_folder = '../output_text/108/2023'
all_data = []

# Duyệt qua từng thư mục con trong thư mục cha (năm)
for month in range(1, 13):
    month_folder = os.path.join(year_folder, f'{month:02d}').replace('\\', '/')
    output_month_folder = os.path.join(output_year_folder, f'{month:02d}').replace('\\', '/')

    # Tạo thư mục cho tháng trong output nếu chưa tồn tại
    os.makedirs(output_month_folder, exist_ok=True)
    
    # Duyệt qua từng file JSON trong thư mục của từng tháng
    for file_name in os.listdir(month_folder):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(month_folder, file_name)

            df = pd.read_excel(file_path)

            if df.empty:
                continue

            df = g(df.copy())

            df = df.fillna(-1)

            # Chuyển đổi timestamp thành dạng ngày giờ có thể đọc được
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.strftime('%Y/%m/%d-%H:%M:%S')

            # Hàm để định dạng dữ liệu cho từng cột
            def format_column(row, column_name):
                if column_name == 'timestamp':
                    return f"Timestamp|{row[column_name]}|x|line_chart"
                else:
                    return f"{column_name}|{row[column_name]}|y|line_chart"

            # Áp dụng hàm định dạng cho mỗi dòng và gộp kết quả thành một dòng duy nhất
            formatted_data_new = df.apply(lambda x: ' '.join([format_column(x, col) for col in df.columns]), axis=1)

            # In ra kết quả (chỉ một phần để kiểm tra)
            print(formatted_data_new.iloc[0])  # In ra kết quả đầu tiên để kiểm tra

            # File path for the output text file
            output_file_path = os.path.join(output_month_folder, file_name.replace('.xlsx', '.txt'))

            formatted_data_string = '\n'.join(formatted_data_new)

            # Writing the single line formatted data to a text file
            with open(output_file_path, 'w') as file:
                file.write(formatted_data_string)

            output_file_path

Timestamp|2023/01/01-00:02:41|x|line_chart room_temp_G-01|24.0|y|line_chart room_temp_G-02|23.0|y|line_chart room_temp_G-06|23.0|y|line_chart room_temp_G-07|24.0|y|line_chart room_temp_L1-06|25.2|y|line_chart room_temp_L1-07|25.2|y|line_chart room_temp_G-04|22.5|y|line_chart room_temp_L1-03|25.5|y|line_chart room_temp_G-08|23.5|y|line_chart room_temp_L1-01|26.2|y|line_chart room_temp_L1-02|26.2|y|line_chart room_temp_G-05|23.0|y|line_chart room_temp_L1-04|26.2|y|line_chart room_temp_L1-05|26.2|y|line_chart room_temp_G-03|23.0|y|line_chart slab_temp_G-01|22.1|y|line_chart slab_temp_G-02|21.8|y|line_chart slab_temp_G-06|22.0|y|line_chart slab_temp_G-07|21.8|y|line_chart slab_temp_L1-06|23.6|y|line_chart slab_temp_L1-07|23.6|y|line_chart slab_temp_G-04|21.2|y|line_chart slab_temp_L1-03|23.0|y|line_chart slab_temp_G-08|23.0|y|line_chart slab_temp_L1-01|22.9|y|line_chart slab_temp_L1-02|22.9|y|line_chart slab_temp_G-05|21.7|y|line_chart slab_temp_L1-04|22.8|y|line_chart slab_temp_L1-05|22.8

In [ ]:
# # File path for the output text file
# output_file_path = '../dataset/formatted_data.txt'

# # Writing the single line formatted data to a text file
# with open(output_file_path, 'w') as file:
#     file.write(formatted_data_new)

# output_file_path